In [2]:
# ============================================================
# GAME RECOMMENDER SYSTEM (KNN)
# Fully self-contained, GitHub & Colab safe
# No broken widget metadata, no app.py required
# ============================================================

# ---------------------------
# 1. INSTALL REQUIRED LIBS
# ---------------------------
!pip install -q numpy pandas scikit-learn matplotlib ipywidgets

# ---------------------------
# 2. IMPORT LIBRARIES
# ---------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import (
    accuracy_score, precision_score,
    recall_score, f1_score, confusion_matrix
)

import ipywidgets as widgets
from IPython.display import display, clear_output

# ---------------------------
# 3. DATASET (EXTENDED VARIETY)
# ---------------------------
games = [
    ("Call of Duty", ["Shooter", "Action"], "Fast-paced military shooter"),
    ("Valorant", ["Shooter", "Strategy"], "Tactical hero-based shooter"),
    ("Elden Ring", ["RPG", "Adventure"], "Open world fantasy RPG"),
    ("The Witcher 3", ["RPG", "Adventure"], "Story-driven dark fantasy RPG"),
    ("FIFA 24", ["Sports"], "Realistic football simulation"),
    ("Forza Horizon", ["Racing"], "Open world racing experience"),
    ("Minecraft", ["Simulation", "Adventure"], "Creative sandbox survival"),
    ("Civilization VI", ["Strategy"], "Turn-based world domination strategy"),
    ("Resident Evil", ["Horror", "Action"], "Survival horror experience"),
    ("Stardew Valley", ["Simulation", "RPG"], "Relaxing farming RPG"),
    ("Need for Speed", ["Racing", "Action"], "High-speed street racing"),
    ("Assassin's Creed", ["Action", "Adventure"], "Historical open-world action"),
    ("Dark Souls", ["RPG", "Action"], "Challenging dark fantasy combat"),
    ("Among Us", ["Strategy"], "Social deduction multiplayer game"),
    ("The Sims", ["Simulation"], "Life simulation sandbox")
]

df = pd.DataFrame(games, columns=["Game", "Genres", "Description"])
df["Primary_Genre"] = df["Genres"].apply(lambda x: x[0])

# ---------------------------
# 4. FEATURE ENGINEERING
# ---------------------------
mlb = MultiLabelBinarizer()
genre_features = mlb.fit_transform(df["Genres"])

tfidf = TfidfVectorizer(stop_words="english")
desc_features = tfidf.fit_transform(df["Description"]).toarray()

X = np.hstack((genre_features, desc_features))

# ---------------------------
# 5. MODEL TRAINING
# ---------------------------
knn = NearestNeighbors(n_neighbors=3, metric="cosine")
knn.fit(X)

# ---------------------------
# 6. EVALUATION (MANDATORY)
# ---------------------------
y_true = df["Primary_Genre"]
y_pred = []

for i in range(len(X)):
    _, idx = knn.kneighbors([X[i]])
    y_pred.append(df.iloc[idx[0][1]]["Primary_Genre"])

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average="macro", zero_division=0)
recall = recall_score(y_true, y_pred, average="macro", zero_division=0)
f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)
cm = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))

# ---------------------------
# 7. USER INTERFACE
# ---------------------------
genre_dropdown = widgets.Dropdown(
    options=sorted(set(g for genres in df["Genres"] for g in genres)),
    description="Genre:"
)

desc_input = widgets.Text(
    description="Description:",
    placeholder="Optional gameplay description"
)

button = widgets.Button(description="Recommend Game")
output = widgets.Output()

def recommend_game(b):
    with output:
        clear_output()

        genre_vec = mlb.transform([[genre_dropdown.value]])
        desc_vec = tfidf.transform([desc_input.value]).toarray() if desc_input.value else np.zeros((1, desc_features.shape[1]))
        user_input = np.hstack((genre_vec, desc_vec))

        distances, indices = knn.kneighbors(user_input)
        recommendations = df.iloc[indices[0]]

        scores = 1 - distances[0]

        print("🎯 Recommended Games:\n")
        for g, s in zip(recommendations["Game"], scores):
            print(f"{g} — Match Score: {round(s * 100, 2)}%")

        # ---------------------------
        # BAR CHART (SINGLE GRAPH)
        # ---------------------------
        plt.figure()
        plt.bar(recommendations["Game"], scores)
        plt.xticks(rotation=45, ha="right")
        plt.title("Game Recommendation Match Scores")
        plt.ylabel("Similarity Score")
        plt.tight_layout()
        plt.show()

        # ---------------------------
        # MODEL EVALUATION OUTPUT
        # ---------------------------
        print("\n📊 Model Evaluation Metrics:")
        print(f"Accuracy: {round(accuracy, 2)}")
        print(f"Precision: {round(precision, 2)}")
        print(f"Recall: {round(recall, 2)}")
        print(f"F1-Score: {round(f1, 2)}")

        print("\nConfusion Matrix:")
        print(cm)

button.on_click(recommend_game)

display(genre_dropdown, desc_input, button, output)

# ============================================================
# END OF PROJECT
# ============================================================


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.4 MB/s eta 0:00:00


Dropdown(description='Genre:', options=('Action', 'Adventure', 'Horror', 'RPG', 'Racing', 'Shooter', 'Simulati…

Text(value='', description='Description:', placeholder='Optional gameplay description')

Button(description='Recommend Game', style=ButtonStyle())

Output()